#Speech Emotion Recognition with MLP Classifier



#Dataset
The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS) 

---
File khusus audio

File hanya audio dari semua aktor (01-24) tersedia sebagai dua file zip terpisah (masing-masing ~ 200 MB):

File ucapan (Audio_Speech_Actors_01-24.zip, 215 MB) berisi 1440 file: 60 percobaan per aktor x 24 aktor = 1440. File lagu (Audio_Song_Actors_01-24.zip, 198 MB) berisi 1012 file: 44 percobaan per aktor x 23 aktor = 1012.

Jumlah = 2452

Pada percobaan model kali ini hanya file ucapan yang digunakan.

---

# Import Paket yang diperlukan

In [4]:
import soundfile # untuk membaca file audio
import numpy as np
import librosa # untuk mengekstrak fitur ucapan
import glob
import os
import pickle # untuk menyimpan model setelah pelatihan
from sklearn.model_selection import train_test_split # untuk pelatihan dan pengujian
from sklearn.neural_network import MLPClassifier # multi-layer perceptron model
from sklearn.metrics import accuracy_score # untuk mengukur seberapa baik model bekerja
from sklearn.metrics import classification_report # untuk mengklasifikasi laporan kerja model
from sklearn.metrics import confusion_matrix # untuk konfusi matriks

Tentukan fungsi extract_feature untuk mengekstrak fitur mfcc, chroma, mel, kontras, dan tonnetz dari file suara.

* mfcc: Mel Frequency Cepstral Coefficient, mewakili spektrum daya jangka pendek suatu suara
* chroma: Berkaitan dengan 12 kelas nada yang berbeda
* mel: Frekuensi Spektogram Mel
* kontras: perbedaan yang nyata apabila diperbandingkan
* tonnetz: harmoni nada

misalnya:
`fitur = extract_feature (jalur, mel = True, mfcc = True, contrast = True, tonnetz = True)`

In [5]:
def extract_feature(file_name, **kwargs):
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

definisikan kamus untuk menampung angka dan emosi yang tersedia dalam kumpulan data RAVDESS, dan daftar untuk menampung semua 8 emosi- netral, tenang, bahagia, sedih, marah, takut, jijik, terkejut.

In [6]:
# all emotions on RAVDESS dataset
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions ( feel free to tune this on your need )
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "neutral",
    "happy"
}

# Muat data dan ekstrak fitur untuk setiap file suara

In [7]:
def load_data(test_size=0.2):
    X, y = [], []
    for file in glob.glob("C:\\Users\\#Ssc_SMAN9BLK\\ravdess data\\Actor_*\\*.wav"):
        # get the base name of the audio file
        basename = os.path.basename(file)
        # get the emotion label
        emotion = int2emotion[basename.split("-")[2]]
        # we allow only AVAILABLE_EMOTIONS we set
        if emotion not in AVAILABLE_EMOTIONS:
            continue
        # extract speech features
        features = extract_feature(file, mfcc=True, chroma=True, mel=True, contrast=True, tonnetz=True)
        # add to data
        X.append(features)
        y.append(emotion)
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

# Pisahkan Set Data
Membagi data menjadi data pelatihan dan data pengujian! data pengujian 25% dari semuanya dan gunakan fungsi load_data untuk mengatur pengukuran.

In [8]:
# load RAVDESS dataset, 75% training 25% testing
X_train, X_test, y_train, y_test = load_data(test_size=0.25)

#set data pelatihan dan pengujian:

In [9]:
#tampilkan bentuk set data pelatihan dan pengujian
print((X_train.shape[0], X_test.shape[0]))

(504, 168)


# Jumlah fitur yang diekstrak.

In [10]:
# jumlah sampel dalam data pelatihan
print("[+] Jumlah sampel data pelatihan:", X_train.shape[0])
# jumlah sampel dalam data pengujian
print("[+] Jumlah sampel data pengujian:", X_test.shape[0])
# jumlah fitur yang digunakan
# ini adalah vektor fitur yang diekstrak 
# menggunakan extract_features() function
print("[+] Jumlah fitur:", X_train.shape[1])

[+] Jumlah sampel data pelatihan: 504
[+] Jumlah sampel data pengujian: 168
[+] Jumlah fitur: 193


# MLP Classifier

In [11]:
# model terbaik, ditentukan oleh pencarian grid
model_params = {
    'alpha': 0.01,
    'batch_size': 300,
    'epsilon': 1e-08, 
    'hidden_layer_sizes': (300,), 
    'learning_rate': 'adaptive', 
    'max_iter': 500, 
}

#menginisialisasi MLP classifier

In [12]:
# menginisialisasi Multi Layer Perceptron classifier
# dengan parameter terbaik
model = MLPClassifier(**model_params)

#lakukan pelatihan pada model.

In [13]:
# pelatihan model
print("[*] Training the model...")
model.fit(X_train, y_train)

[*] Training the model...


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.01, batch_size=300, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

# Memprediksi keakuratan model kita

Mari kita prediksi nilai untuk data pengujian. Ini memberi kita y_pred (emosi yang diprediksi untuk fitur dalam set data pengujian).

In [16]:
# memprediksi 25% data untuk mengukur seberapa baik kita
y_pred = model.predict(X_test)
y_pred

array(['happy', 'angry', 'happy', 'sad', 'happy', 'angry', 'happy',
       'angry', 'angry', 'happy', 'sad', 'neutral', 'neutral', 'neutral',
       'sad', 'sad', 'happy', 'angry', 'sad', 'sad', 'angry', 'happy',
       'neutral', 'happy', 'sad', 'happy', 'angry', 'angry', 'neutral',
       'angry', 'sad', 'sad', 'sad', 'angry', 'neutral', 'sad', 'sad',
       'happy', 'neutral', 'neutral', 'sad', 'angry', 'angry', 'neutral',
       'angry', 'sad', 'happy', 'neutral', 'neutral', 'happy', 'sad',
       'neutral', 'happy', 'happy', 'sad', 'angry', 'angry', 'happy',
       'angry', 'angry', 'neutral', 'sad', 'sad', 'sad', 'happy',
       'neutral', 'neutral', 'angry', 'happy', 'angry', 'happy', 'happy',
       'sad', 'angry', 'angry', 'happy', 'angry', 'happy', 'sad', 'angry',
       'sad', 'sad', 'happy', 'angry', 'happy', 'neutral', 'sad', 'sad',
       'sad', 'angry', 'sad', 'neutral', 'sad', 'angry', 'happy', 'angry',
       'angry', 'sad', 'sad', 'happy', 'neutral', 'sad', 'angry', '

Untuk menghitung akurasi model kita, kita akan memanggil fungsi akurasi_score () yang kita impor dari sklearn.

In [17]:
# menghitung akurasi
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 82.14%


#Laporan klasifikasi

In [18]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       angry       0.95      0.89      0.92        61
       happy       0.74      0.78      0.76        41
     neutral       0.71      0.81      0.76        21
         sad       0.80      0.78      0.79        45

    accuracy                           0.82       168
   macro avg       0.80      0.81      0.80       168
weighted avg       0.83      0.82      0.82       168



# Confusion Matrix

In [19]:
matrix = confusion_matrix(y_test,y_pred)
print (matrix)

[[54  5  1  1]
 [ 2 32  1  6]
 [ 1  1 17  2]
 [ 0  5  5 35]]


In [20]:
# simpan modelnya
# buat direktori hasil jika belum ada
if not os.path.isdir("result"):
    os.mkdir("result")

pickle.dump(model, open("result/mlp_classifier.model", "wb"))

# Sekian